In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
df_cpc_com = pd.read_csv('../midsave/cpc_com.csv', dtype={'code_com': str})
df_xwalk = pd.read_csv('../midsave/xwalk.csv', dtype={'code_com': str, 'code_epci': str, 'code_dep': str})
df_commune = pd.read_excel('../external_data/commune_level_demographic_data_2.xlsx', dtype={'code_com': str}).replace(to_replace=r'N/A - résultat non disponible', value=np.nan, regex=True)
df_inter_com = pd.read_excel('../external_data/inter_communalite_demographic_data_2.xlsx', dtype={'code_epci': str}).replace(to_replace=r'N/A - résultat non disponible', value=np.nan, regex=True)
df_department = pd.read_excel('../external_data/department_level_demographic_data_2.xlsx', dtype={'code_dept': str}).rename(columns = {'code_dept':'code_dep'}).replace(to_replace=r'N/A - résultat non disponible', value=np.nan, regex=True)
df_political = pd.read_csv('../external_data/presidential_elections_reformatted.csv', delimiter=';', dtype={'code_com': str})
df_poi = pd.read_csv('../midsave/poi.csv', dtype={'code_com': str})
df_gtrends = pd.read_csv('../midsave/gtrends_dep.csv', dtype={'code_dep': str})

In [ ]:
df_cpc_com.head()

In [ ]:
df_xwalk.head()

In [ ]:
df_commune.head()

In [ ]:
df_inter_com.head()

In [ ]:
df_department.head()

In [ ]:
df_political.head()

Harmonize coding schemes

In [ ]:
dept_to_remove = ['2A', '2B', 'ZA', 'ZB', 'ZC', 'ZD', 'ZM', 'ZN',
       'ZP', 'ZS', 'ZW', 'ZX', 'ZZ']

df_political = df_political[~(df_political['code_dept'].isin(dept_to_remove))]

df_political['code_dept'] = df_political['code_dept'].astype(int) + 100
df_political['code_com'] = df_political['code_com'].astype(int) + 1000

df_political['code_dept'] = df_political['code_dept'].astype(str).str[1:]
df_political['code_com'] = df_political['code_com'].astype(str).str[1:]

In [ ]:
df_political['code_com'] = df_political['code_dept'] + df_political['code_com']

In [ ]:
df_political.head()

In [ ]:
df_poi.head()

In [ ]:
df_gtrends.head()

In [ ]:
df = (df_cpc_com[['code_com', 
                'log_cpc_per_1000', 'log_yt_per_1000', 'log_wa_per_1000', 'log_tor_per_1000', 
                'cpc_per_1000', 'yt_per_1000', 'wa_per_1000', 'tor_per_1000', 
                'sv_17_21', 'sv_com_17_21']]
      .merge(df_xwalk[['code_com', 'code_epci', 'code_dep', 'code_reg']].drop_duplicates(), on = 'code_com', how = 'left')
      .merge(df_commune[['code_com', 'Densité de population (historique depuis 1876) 2020', 'Part des pers. de 15 ans ou + célibataires 2020', 'Usage de stupéfiants (taux) 2022']], on = 'code_com', how = 'left')
      .merge(df_inter_com[['code_epci', 'Taux de pauvreté 2020']], on = 'code_epci', how = 'left')
      .merge(df_department[['code_dep', 'Taux de chômage annuel moyen 2022']], on = 'code_dep', how = 'left')
      .merge(df_political[['code_com', '% Exp/Ins', '% Voix/Ins_lepen', '% Voix/Ins_macron']], on = 'code_com', how = 'left')
      .merge(df_poi, on = 'code_com', how = 'left')
      .merge(df_gtrends[['code_dep', 'PC1', 'PC2', 'PC3', 'lin_1', 'lin_2', 'lin_3']], on = 'code_dep', how = 'left'))

In [ ]:
df.shape

In [ ]:
df.dropna().shape

In [ ]:
df.to_csv("../midsave/regression.csv", index=False)